<a href="https://colab.research.google.com/github/yannick10800/Python/blob/main/CalculBETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation package

In [ ]:
pip install yfinance

     |████████████████████████████████| 6.3MB 13.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.61-py2.py3-none-any.whl size=23818 sha256=b0a535962aab3c025ecdc899bfaf42a20ac94af4ff096bf174872c65309f7116
  Stored in directory: /root/.cache/pip/wheels/9c/b3/31/b5f2bcf2e3fd3f42e5ca1a3d20ada27eae8cbda7de75a5a2a0
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
from pandas_datareader import data
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
from pandas.stats.api import ols

ModuleNotFoundError: ignored

# Recuperation des données 

https://tinytrader.io/how-to-calculate-stock-beta-automatically-with-python/ il n'y a plus de OLS, il faut le recupérer dans statsmodels https://www.statsmodels.org/dev/

In [ ]:
# stock and market index tickers
stock_symbol = 'COIN'
#mrkt_symbol = raw_input( '''Enter ticker symbol for market index (e.g., ^GSPC): ''').upper()
mrkt_symbol = 'GSPC''


NameError: ignored

In [ ]:
# set time period for historical prices
end_time = time.strftime('%m/%d/%Y')
start_time = end_time[:6] + str(int(end_time[-4:]) - 20)


In [ ]:
# retreive historical prices for stock
#stock = data.DataReader(stock_symbol,data_source='google', start=start_time, end=end_time)
#stock=yf.ticker('coin')
monticker='BTC-USD'
#recuperer les datas
databtc=yf.Ticker(monticker)
#recuperer l'historique des prix
dataBTC = databtc.history(period='1d',start='2014-09-17',end=datejour)

NotImplementedError: ignored

In [ ]:
# simplify data sets
stock_simple = stock.rename(columns={'Close':'Stock'})
mrkt_simple = mrkt.rename(columns={'Close':'Mrkt'})


In [ ]:
# combine stock and market index closing prices
data_simple = pd.concat([mrkt_simple, stock_simple],axis=1, join='inner')[['Mrkt', 'Stock']]


In [ ]:
# keep only values by month
data_month = data_simple.asfreq('M', method='ffill')


In [ ]:
# calculate percent change of closing prices
data_month['MrktPctChg'] = (data_month.Mrkt /data_month.Mrkt.shift(1) - 1)
data_month['StockPctChg'] = (data_month.Stock /data_month.Stock.shift(1) - 1)


# Calcul linearisition

In [ ]:
# plot monthly returns
scat_data = data_month.tail(59)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
ax1.scatter(scat_data['MrktPctChg'],scat_data['StockPctChg'],label='Monthly Returns', color='blue',edgecolors='none', alpha=0.5)
ax1.grid(True)


In [ ]:
# set x and y coordinate limits
x_corr = (-0.05, 0.05)
y_corr = (-0.05, 0.05)

In [ ]:
# draw lines for x and y axes
ax1.plot(x_corr, (0, 0), color='black', alpha=0.4)
ax1.plot((0, 0), y_corr, color='black', alpha=0.4)


NameError: ignored

In [ ]:
# reset x and y coordinate limits back to original
ax1.set_xlim(x_corr[0], x_corr[1])
ax1.set_ylim(y_corr[0], y_corr[1])


In [ ]:
# set axis labels
ax1.set_xlabel('Market Index Monthly Returns')
ax1.set_ylabel(stock_symbol + ' Monthly Returns')


In [ ]:
# create an array of x (i.e. market index) values
x = np.linspace(x_corr[0] + 0.005, x_corr[1] - 0.005)


In [ ]:
# run regression to calculate beta
reg_month = ols(y=scat_data['StockPctChg'],x=scat_data['MrktPctChg'])


In [ ]:
# define function for regression equation
def month(x):
return reg_month.beta[1] + reg_month.beta[0] * x

In [ ]:
# plot beta for monthly returns
ax1.plot(x, month(x), color='magenta', linestyle='-',
linewidth=2, label='Beta', alpha=0.8)


In [ ]:
# add legend
ax1.legend(loc='upper center',bbox_to_anchor=(0.5, -0.075), ncol=2, fontsize='large')


In [ ]:
# add Beta and R2 stats to plot
ax1.text(-0.038, 0.042,'Beta: ' + str(np.round(reg_month.beta[0], 3)),fontsize=12)
ax1.text(-0.038, 0.035,'R2: ' + str(np.round(reg_month.r2, 3)),fontsize=12)


In [ ]:
# calculate rolling betas
for date in data_month.index[1:]:
  data = data_month.loc[:date].tail(59)
  reg = ols(y=data['StockPctChg'], x=data['MrktPctChg'])
  data_month.loc[date, 'Beta'] = reg.beta[0]


In [ ]:
# plot only betas with 60 data points
if len(data_month) &amp;amp;amp;amp;gt; 60:
  to_plot = data_month[data_month.index[60]:]
else:
  to_plot = data_month
  ax2.plot(to_plot.index, to_plot.Beta, color='blue',
  linewidth=2, alpha=0.5, label='5-year Rolling Beta')
  ax2.grid(True)


In [ ]:
# get x coordinate limits
x_corr = ax2.get_xlim()
# draw lines for x axis
ax2.plot(x_corr, (1, 1), color='black', alpha=0.4)
# add y label
ax2.set_ylabel(stock_symbol + ' Beta')
# add legend
ax2.legend(loc='upper center',
bbox_to_anchor=(0.5, -0.075), ncol=1, fontsize='large')
# display plot
fig.show()
